## LangChain: Agents ##

### Outline: ###
- Using built in LangChain tools: DuckDuckGo search and Wikipedia
- Defining your own tools

- Sometimes people think of a large language model as a knowledge store, as if it's learned to memorize a lot of information, maybe off the internet, so when you ask it a question, it can answer the question. 
- A even more useful way to think of a large language model is sometimes as a **reasoning engine**, i
    - you can give it chunks of text or other sources of information. 
    - the large language model, LLM, will use this background knowledge that's learned off the internet, but to use the new information you give it to help you answer questions or reason through content or decide even what to do next. 
- that's what LangChain's Agents framework helps you to do. 

Agents 


In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
os.environ['OPENAI_API_KEY']='xxxxx'

In [3]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

### Built-in LangChain tools ###

In [5]:
#!pip install -U wikipedia

In [6]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

In [7]:
llm = ChatOpenAI(temperature=0, model=llm_model)

**load two tools: "llm-math" and "wikipedia"**
- llm-math tool is a chain itself, it use a language model and conjuntion with a calculator to do math problems
- wikipedia tool is an API that connect to wikipedia, allowing you to run search wikipedia and return result

In [8]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

**Initialize the agent**
- CHAT_ZERO_SHOT_REACT_DESCRIPTION
    - Chat: This is optimized to work with chat models. 
    - React: is a prompting strategy that elicits better thoughts from a language model. 
- handle_parsing_errors=True
    - This is useful when LLM may output something that is not able to be parsed into an action input or desired output

In [9]:
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [10]:
agent("What is the 25% of 300?")



> Entering new AgentExecutor chain...
Thought: We need to calculate 25% of 300, which involves multiplication and division.

Action:
```
{
  "action": "Calculator",
  "action_input": "0.25*300"
}
```


Observation: Answer: 75.0
Thought:We have the answer to the question.

Final Answer: 75.0

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '75.0'}

### Wikipedia example ###

In [11]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question) 



> Entering new AgentExecutor chain...
Thought: I should use Wikipedia to search for Tom M. Mitchell's books.
Action:
```
{
  "action": "Wikipedia",
  "action_input": "Tom M. Mitchell"
}
```

Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past President of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Int

### Python Agent ###
- like copilot

In [12]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [13]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [14]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 



> Entering new AgentExecutor chain...
I can use the sorted() function to sort the list of customers by last name and then first name. I will need to provide a key function to sorted() that returns a tuple of the last name and first name in that order.
Action: Python REPL
Action Input:
```
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted_customers = sorted(customers, key=lambda x: (x[1], x[0]))
for customer in sorted_customers:
    print(customer)
```
Observation: ['Jen', 'Ayai']
['Lang', 'Chain']
['Harrison', 'Chase']
['Elle', 'Elem']
['Trance', 'Former']
['Geoff', 'Fusion']
['Dolly', 'Too']

Thought:The customers are now sorted by last name and then first name. 
Final Answer: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

> Finished chain.


"[['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]"

The format Action and Action Input are slightly different with before, because it use different agent
- the Action: Python REPL
- Action Input: is a bunch of python code.

#### View detailed outputs of the chains ####

In [15]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nI

[llm/end] [1:chain:AgentExecutor > 5:chain:LLMChain > 6:llm:ChatOpenAI] [1.33s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The customers are now sorted by last name and then first name. \nFinal Answer: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]",
        "generation_info": null,
        "message": {
          "content": "The customers are now sorted by last name and then first name. \nFinal Answer: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]",
          "additional_kwargs": {},
          "example": false
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 526,
      "completion_tokens": 68,
      "total_tokens": 594
    },
    "model_name": "gpt-3.5-turbo-0301"
  }
}
[chain/end] [1:chain:AgentExecutor > 5:chain:LLMChain] [

[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
- 2:chain:LLMChain]: is the llmchain the agent is using
    - it is the combination of prompt and llm
    - at this point, it only got 
        - "input"
        - "agent_scratchpad": "" #empty here
        - "stop": tell the llm when to stop
        
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
- at this level, we see the exactly call to the llm
    - it includes fully formatted prompts
    
[llm/end] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] [2.91s] Exiting LLM run with output:
- exiting with "generation" from the language model, which is the model's response to the input prompt.

[chain/end] [1:chain:AgentExecutor > 2:chain:LLMChain] [2.91s] Exiting Chain run with output:
- wrapped and exit with output "text"(instruction)  as input to tool

[tool/start] [1:chain:AgentExecutor > 4:tool:Python REPL] Entering Tool run with input:
- we can see the input to the tool: python code

[tool/end] [1:chain:AgentExecutor > 4:tool:Python REPL] [1.198ms] Exiting Tool run with output:
- the output is the sorted customer list
- it has this list because the code in the tool start:
    ```python
    for customer in sorted_customers:
    print(customer)
    ```

[chain/start] [1:chain:AgentExecutor > 5:chain:LLMChain] Entering Chain run with input:
- the llmchain is the agent
    - input:
    - "agent_scratchpad" : now there are some new values here
        - a combination of previous generations + the tool output
        - we packing these back in so the language model can understand what happen previously and use it to reason about what to do next

[llm/start] [1:chain:AgentExecutor > 5:chain:LLMChain > 6:llm:ChatOpenAI] Entering LLM run with input:
- the llmchain realized that it finished it's job

[llm/end] [1:chain:AgentExecutor > 5:chain:LLMChain > 6:llm:ChatOpenAI] [1.33s] Exiting LLM run with output:
- llmchain realized that the job is done and set it's final answer

[chain/end] [1:chain:AgentExecutor > 5:chain:LLMChain] [1.33s] Exiting Chain run with output:
- The agentExecutor realized the job is done

[chain/end] [1:chain:AgentExecutor] [4.25s] Exiting Chain run with output:
- The agentExcutor realized the job is done and send the output and exit

### Define your own tool ###
A big power of agents is that you can **connect it to your own sources of information, your own APIs, your own data**

Create a custom tool so that you can connect it to whatever you want.
- make a tool that tell what the current date is
    - tool decorator
        - This can be applied to any function and it turns it into a tool that LangChain can use
    - write a function called "time", which takes in any text string. And it's going to return today's date by calling date time. 
    - In addition to the name of the function, we're also going to write a really detailed doc string. 
        - this is what the agent will use to know when and how it should call this tool 
        - For example, here we say that, "The input should always be an empty string.". That's because we don't use it. 
        - If we have more strang requirements on what the input should be, for example, if we have a function that should always take in a search query or a SQL statement, you'll want to make sure to mention that here

In [17]:
#!pip install DateTime

In [18]:
from langchain.agents import tool
from datetime import date

In [19]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

adding the time tool to the list of exiting tools

In [20]:
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [21]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")



> Entering new AgentExecutor chain...
Thought: I need to use the `time` tool to get today's date.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2023-12-10
Thought:I have successfully retrieved today's date using the `time` tool.
Final Answer: Today's date is 2023-12-10.

> Finished chain.
